In [1]:
# !pip install openai

In [5]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [6]:
from openai import OpenAI
client = OpenAI()

In [13]:
system_prompt = '''
    You are an agent who specializes in describing an image that will be recreated by an artist. Describe all the elements of the image.

    Return keywords in the format of an array of strings, like this:
    ['card', 'woman', 'wind']
'''

import base64

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
def analyze_image(img_path, send_image_type):
    if send_image_type == 'file_image':
        # image_path is a local path
        content = f"data:image/jpeg;base64,{encode_image(img_path)}"
    elif send_image_type == 'url':
        # image_path is a url link
        content = img_path

    response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": content,
                    }
                },
            ],
        }
    ],
        max_tokens=300,
        top_p=0.1
    )

    return response.choices[0].message.content

In [20]:
images = []
descritions = []

for image_path in os.listdir('CARTA-SAKURA'):
    describe = analyze_image('CARTA-SAKURA/'+image_path, 'file_image')

    images.append('CARTA-SAKURA/'+image_path)
    descritions.append(describe)

In [59]:
print(descritions[0])

The image features a stylized character depicted in a tarot card format. The character, a woman, has long, flowing hair and is adorned with a crown-like headpiece that has multiple points. She stands with her arms crossed in front of her, holding a small orb in one hand and a cloud-like shape in the other. 

The background is a gradient of purple, and at the top, there is a sun symbol with rays extending outward. The bottom of the card features a banner with the text "THE LIGHT." There are decorative star motifs on either side of the character, enhancing the mystical theme of the card.

Keywords: ['tarot', 'woman', 'light', 'crown', 'sun', 'stars', 'purple', 'orb', 'cloud']


In [ ]:
import pandas as pd
df = pd.DataFrame({
    "images":images,
    "descritions":descritions
})

In [ ]:
from datasets import Dataset, DatasetDict
from PIL import Image

In [43]:
hf_dataset = Dataset.from_pandas(df)

In [47]:
from datasets import Dataset, Image

In [48]:
hf_dataset = hf_dataset.cast_column("images", Image())

In [53]:
from huggingface_hub import login

In [55]:
login(token=os.environ['HF_TOKEN'])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\Wayn\.cache\huggingface\token
Login successful


In [57]:
hf_dataset.push_to_hub("WillzWayn/sakura-card-captor-cards")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.94s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/WillzWayn/sakura-card-captor-cards/commit/9389632be9f1842e622fb6263f833def8995e72f', commit_message='Upload dataset', commit_description='', oid='9389632be9f1842e622fb6263f833def8995e72f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/WillzWayn/sakura-card-captor-cards', endpoint='https://huggingface.co', repo_type='dataset', repo_id='WillzWayn/sakura-card-captor-cards'), pr_revision=None, pr_num=None)